In [2]:
#匯入牌組資料
import pandas as pd
card_data=pd.read_excel("塔羅牌資料.xlsx")

In [3]:
#初始抽牌template
start={
    "type": "buttons",
    "thumbnailImageUrl": "https://i.imgur.com/NQPjz4s.jpg",
    "imageAspectRatio": "square",
    "imageSize": "contain",
    "imageBackgroundColor": "#FFFFFF",
    "title": "小戴塔羅、鐵口直斷",
    "text": "點選下方開始抽牌",
    "actions": [
      {
        "type": "postback",
        "label": "抽牌",
        "data": "抽牌"
      }
    ]
  }

In [4]:
import random as rd
pn=["正","逆"]#positive or negative
#抽牌結果[[牌號1,正逆],[牌號2,正逆],[牌號3,正逆]] 1正2逆
def draw_cards():
    global card_numbers
    global card_list
    global card
    card_list=[]
    card_numbers = rd.sample(range(0, 78), 3)#抽三張不重複
    for card in card_numbers:
        card_list.append([card,rd.randint(1,2)])
    print("使用者抽牌:",card_list)
    #抽牌結果
    card={
  "type": "carousel",
  "contents":[
    {
      "type": "bubble",
      "direction": "ltr",
      "header": {
        "type": "box",
        "layout": "baseline",
        "contents": [
          {
            "type": "text",
            "text": "過去",
            "color": "#FF9D00FF",
            "align": "center"
          }
        ]
      },
      "hero": {
        "type": "image",
        "url": card_data.iat[card_list[0][0],3],
        "size": "full",
        "aspectRatio": "1.51:1",
        "aspectMode": "fit"
      },
      "body": {
        "type": "box",
        "layout": "vertical",
        "contents": [
          {
            "type": "text",
            "text": card_data.iat[card_list[0][0],0]+pn[card_list[0][1]-1],
            "align": "center"
          }
        ]
      },
      "footer": {
        "type": "box",
        "layout": "horizontal",
        "contents": [
          {
            "type": "button",
            "action": {
              "type": "postback",
              "label": "觀看牌義",
              "data": str(card_list[0][0])
            }
          }
        ]
      }
    },
    {
      "type": "bubble",
      "direction": "ltr",
      "header": {
        "type": "box",
        "layout": "baseline",
        "contents": [
          {
            "type": "text",
            "text": "現在",
            "color": "#FF9D00FF",
            "align": "center"
          }
        ]
      },
      "hero": {
        "type": "image",
        "url": card_data.iat[card_list[1][0],3],
        "size": "full",
        "aspectRatio": "1.51:1",
        "aspectMode": "fit"
      },
      "body": {
        "type": "box",
        "layout": "vertical",
        "contents": [
          {
            "type": "text",
            "text": card_data.iat[card_list[1][0],0]+pn[card_list[1][1]-1],
            "align": "center"
          }
        ]
      },
      "footer": {
        "type": "box",
        "layout": "horizontal",
        "contents": [
          {
            "type": "button",
            "action": {
              "type": "postback",
              "label": "觀看牌義",
              "data": str(card_list[1][0])
            }
          }
        ]
      }
    },
    {
      "type": "bubble",
      "direction": "ltr",
      "header": {
        "type": "box",
        "layout": "baseline",
        "contents": [
          {
            "type": "text",
            "text": "未來",
            "color": "#FF9D00FF",
            "align": "center",
          }
        ]
      },
      "hero": {
        "type": "image",
        "url": card_data.iat[card_list[2][0],3],
        "size": "full",
        "aspectRatio": "1.51:1",
        "aspectMode": "fit"
      },
      "body": {
        "type": "box",
        "layout": "vertical",
        "contents": [
          {
            "type": "text",
            "text": card_data.iat[card_list[2][0],0]+pn[card_list[2][1]-1],
            "align": "center"          }
        ]
      },
      "footer": {
        "type": "box",
        "layout": "horizontal",
        "contents": [
          {
            "type": "button",
            "action": {
              "type": "postback",
              "label": "觀看牌義",
              "data": str(card_list[2][0])
            }
          }
        ]
      }
    }
  ]
}

In [ ]:
from flask import Flask, request
from linebot import LineBotApi
from linebot.models import FlexSendMessage, TextSendMessage,TemplateSendMessage
app = Flask(__name__)

# LINE 聊天機器人的基本資料
line_bot_api = LineBotApi('0MTwO+uLC9YQAIsSUu88agU/9o/MxFFa83qF0OPczd/ivnMam25adYe0lMDl7E+lTadbxeC3wYZbjPqTlZM1Ea3aP5kYuxIVTdHHpac8beN7ZA24XnGckQYWTi0Q4Ro9URFdR5lN4BGgM6Zc+W0D1wdB04t89/1O/w1cDnyilFU=')
@app.route("/", methods=['POST'])
def fortune():
    data = request.get_json()
    print(data)
    if len(data['events'])==0:#驗證模式events是空的
        print("webhook驗證OK")
        return 'Verify OK',200
    elif data['events'][0]['type']=="message" and len(data['events'])==1:#任何文字訊息皆發送啟動template
        if data['events'][0]['message']["text"]!="開始抽牌":    
            line_bot_api.reply_message(data['events'][0]['replyToken'],
            TemplateSendMessage(alt_text="123", template=start))
    if data['events'][0]['type']=="postback":
        if data['events'][0]["postback"]["data"]=="抽牌":#使用者點選抽牌
            draw_cards()
            line_bot_api.reply_message(data['events'][0]['replyToken'],
            FlexSendMessage(alt_text="456",contents=card))
        else:#使用者點選觀看牌義
            card_num=int(data['events'][0]["postback"]["data"])
            line_bot_api.reply_message(data['events'][0]['replyToken'],
            TextSendMessage(text=card_data.iat[card_num,card_list[card_numbers.index(card_num)][1]]))
    return 'OK',200
if __name__ == "__main__":
    app.run(port=3000)#需與NGROK的通道阜相同

 * Serving Flask app '__main__'
 * Debug mode: off


C:\Users\hp\AppData\Local\Temp\ipykernel_636\3094302790.py:7: LineBotSdkDeprecatedIn30: Call to deprecated class LineBotApi. (Use v3 class; linebot.v3.<feature>. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api = LineBotApi('0MTwO+uLC9YQAIsSUu88agU/9o/MxFFa83qF0OPczd/ivnMam25adYe0lMDl7E+lTadbxeC3wYZbjPqTlZM1Ea3aP5kYuxIVTdHHpac8beN7ZA24XnGckQYWTi0Q4Ro9URFdR5lN4BGgM6Zc+W0D1wdB04t89/1O/w1cDnyilFU=')
 * Running on http://127.0.0.1:3000
Press CTRL+C to quit


{'destination': 'Ud599aec203f3d52e84ab5ea488f2c69e', 'events': [{'type': 'message', 'message': {'type': 'text', 'id': '501704463732703690', 'quoteToken': 'wNnJsgwl4yxwix3hBjF2Yuk2qQk7_2n1waypqHaca2_XwahJMNn49rWY7F4ubk5bQO2Qp10dG2FFL-aT2obdlx4jkMXmwg90fUQhXYit0aO_DUpQSWRzhwJkXarjoRxystpnkJQq3tEdP_796dihHQ', 'text': '123'}, 'webhookEventId': '01HT9PWQFAEM9RTHT030NH0RFB', 'deliveryContext': {'isRedelivery': False}, 'timestamp': 1711870467448, 'source': {'type': 'user', 'userId': 'U045e09b64fee492ed95edeab7c1481bd'}, 'replyToken': 'b13150fc69004f5bb54a8399a568b649', 'mode': 'active'}]}


C:\Users\hp\AppData\Local\Temp\ipykernel_636\3094302790.py:17: LineBotSdkDeprecatedIn30: Call to deprecated method reply_message. (Use 'from linebot.v3.messaging import MessagingApi' and 'MessagingApi(...).reply_message(...)' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api.reply_message(data['events'][0]['replyToken'],
127.0.0.1 - - [31/Mar/2024 15:34:27] "POST / HTTP/1.1" 200 -


{'destination': 'Ud599aec203f3d52e84ab5ea488f2c69e', 'events': [{'type': 'postback', 'postback': {'data': '抽牌'}, 'webhookEventId': '01HT9PWXZQMXYMMKX2R1QMN0WF', 'deliveryContext': {'isRedelivery': False}, 'timestamp': 1711870473730, 'source': {'type': 'user', 'userId': 'U045e09b64fee492ed95edeab7c1481bd'}, 'replyToken': '9c435089599b4027ac9a4557a1db4cc1', 'mode': 'active'}]}
使用者抽牌: [[7, 1], [9, 1], [56, 2]]


...

127.0.0.1 - - [31/Mar/2024 15:35:28] "POST / HTTP/1.1" 200 -


{'destination': 'Ud599aec203f3d52e84ab5ea488f2c69e', 'events': [{'type': 'postback', 'postback': {'data': '抽牌'}, 'webhookEventId': '01HT9PZ0BVDPGFYMJN25SCGDR1', 'deliveryContext': {'isRedelivery': False}, 'timestamp': 1711870541831, 'source': {'type': 'user', 'userId': 'U045e09b64fee492ed95edeab7c1481bd'}, 'replyToken': 'e5e8e22877ad4b5d8e9124cd2f6e45c1', 'mode': 'active'}]}
使用者抽牌: [[77, 1], [60, 2], [46, 1]]


127.0.0.1 - - [31/Mar/2024 15:35:42] "POST / HTTP/1.1" 200 -
[2024-03-31 15:36:15,818] ERROR in app: Exception on / [POST]
Traceback (most recent call last):
  File "C:\Users\hp\anaconda3\Lib\site-packages\flask\app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\hp\anaconda3\Lib\site-packages\flask\app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\hp\anaconda3\Lib\site-packages\flask\app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\hp\anaconda3\Lib\site-packages\flask\app.py", line 1796, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\hp\AppData\Local\Temp\ipykernel_636\3094302790.py", line 27, 

{'destination': 'Ud599aec203f3d52e84ab5ea488f2c69e', 'events': [{'type': 'postback', 'postback': {'data': '56'}, 'webhookEventId': '01HT9Q01T11WEG922NR873MN4H', 'deliveryContext': {'isRedelivery': False}, 'timestamp': 1711870575971, 'source': {'type': 'user', 'userId': 'U045e09b64fee492ed95edeab7c1481bd'}, 'replyToken': '538a7fb452214e068a0414b337635eb5', 'mode': 'active'}]}
{'destination': 'Ud599aec203f3d52e84ab5ea488f2c69e', 'events': [{'type': 'postback', 'postback': {'data': '77'}, 'webhookEventId': '01HT9Q08MZ9TX0XDBM3ZBV1RXX', 'deliveryContext': {'isRedelivery': False}, 'timestamp': 1711870582953, 'source': {'type': 'user', 'userId': 'U045e09b64fee492ed95edeab7c1481bd'}, 'replyToken': '68e75c01b98d4f3d84c6469e3b7f7e0b', 'mode': 'active'}]}


127.0.0.1 - - [31/Mar/2024 15:36:23] "POST / HTTP/1.1" 200 -
